In [7]:
from init import *
#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])
config.main_token='UUOS'
wallet.import_key('test', '5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')
for priv_key in priv_keys:
    wallet.import_key('test', priv_key)

db.reset()
print('done!')

done!


In [239]:
code = r'''
#include <eosio/print.hpp>
#include <eosio/name.hpp>
#include <eosio/action.hpp>
#include <eosio/multi_index.hpp>
#include <eosio/singleton.hpp>
#include <eosio/fixed_bytes.hpp>
#include "eth_account.hpp"

using namespace eosio;
using namespace std;

__attribute__((eosio_wasm_import))
extern "C" int evm_execute(const unsigned char* code, size_t size);

template<typename T>
T unpack_args() {
    size_t raw_args_size = action_data_size();
    check(raw_args_size > 0, "bad args");
    vector<char> raw_args(raw_args_size);
    read_action_data(raw_args.data(), raw_args_size);
    T rec = eosio::unpack<T>(raw_args);
    return rec;
}
   
extern "C" {
    void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
        if (action == "active"_n.value) {
            eth_address address;
            int size = read_action_data(address.data(), address.size());
            eosio::printhex(address.data(), address.size());
            check(size == 20, "bad address");
            bool ret = eth_account_create(address);
            eosio::print(ret);
        } else if (action == "exec"_n.value) {
            int raw_args_size = action_data_size();
            vector<unsigned char> raw_args(raw_args_size);
            read_action_data(raw_args.data(), raw_args_size);
            evm_execute(raw_args.data(), raw_args.size());
        }
    }
}
    
#include "eth_account.cpp"

'''

abi = '''
{
   "version": "eosio::abi/1.0",
   "types": [],
  "structs": [{
      "name": "active",
      "base": "",
      "fields": [
        {"name":"address", "type":"bytes"},
       ]
    }
  ],
  "actions": [{
      "name": "active",
      "type": "active",
      "ricardian_contract": ""
    },{
      "name": "exec",
      "type": "bytes",
      "ricardian_contract": ""
    }
  ],
  "tables": [
  ]
}
'''

publish_cpp_contract('helloworld11', code, abi, includes=['contracts'])
print('done!')

done!


In [240]:
name = 'helloworld11'

address = b'\x11\x22\x33\x44\x55\x66\x77\x88\x99\x00\x11\x22\x33\x44\x55\x66\x77\x88\x99\x00'
#args = {'address':'2c7536E3605D9C16a7a3D7b1898e529396a65c23'}
#args = b'\x11\x22\x33\x44\x55\x66\x77\x88\x99\x00\x11\x22\x33\x44\x55\x66\x77\x88\x99\x00'
#args = {'address':address.hex()}
args = bytes.fromhex('2c7536E3605D9C16a7a3D7b1898e529396a65c23')
try:
    r = eosapi.push_action(name, 'active', args, {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print(e)


2c7536e3605d9c16a7a3d7b1898e529396a65c23false
201


In [265]:
import web3
from solc import compile_source, compile_files

greeter = '''
pragma solidity ^0.4.8;
contract Greeter {
    uint value;
    event onSetValue(uint value);
    event onGetValue(uint value);
    function Greeter() {
        value = 1;
    }

    function getValue2() payable public returns (uint){
//        msg.sender.transfer(1000);
        onGetValue(value);
        return value;
    }

    function getValue() payable public returns (string){
        return "hello,world";
    }

    function setValue(uint v) payable public {
        value = v;
        onSetValue(v);
    }
}
'''

def compile(contract_source_code, main_class):
    compiled_sol = compile_source(contract_source_code) # Compiled source code
    contract_interface = compiled_sol[main_class]

    return contract_interface


main_class = '<stdin>:Greeter'
contract_source_code = greeter
contract_interface = compile(contract_source_code, main_class)

for abi in contract_interface['abi']:
    print(abi)
#print(contract_interface['bin'])

import web3
transaction = {
     'from': '0x2c7536E3605D9C16a7a3D7b1898e529396a65c23',
     'data': contract_interface['bin'],
     'gas': 2000000,
     'gasPrice': 23,
     'nonce': 0,
}

key = '0x4c0883a69102937d6231471b5dbb6204fe5129617082792ae468d01a3f362318'
signed = web3.eth.Account.signTransaction(transaction, key)
#print(signed.rawTransaction)
#for c in signed.rawTransaction:
#    print('0x%02x,'%(c,), end='')

name = 'helloworld11'
try:
    r = eosapi.push_action(name, 'exec', signed.rawTransaction, {name:'active'})
    res = r['processed']['action_traces'][0]['console']
    print(r['processed']['elapsed'])

    import json
    print(res)
    res = json.loads(res)
    new_address = web3.Web3.toChecksumAddress(res['newAddress'])
    print(new_address)
    print(res['output'])
except Exception as e:
    print(e)


{'constant': False, 'inputs': [], 'name': 'getValue', 'outputs': [{'name': '', 'type': 'string'}], 'payable': True, 'stateMutability': 'payable', 'type': 'function'}
{'constant': False, 'inputs': [{'name': 'v', 'type': 'uint256'}], 'name': 'setValue', 'outputs': [], 'payable': True, 'stateMutability': 'payable', 'type': 'function'}
{'constant': False, 'inputs': [], 'name': 'getValue2', 'outputs': [{'name': '', 'type': 'uint256'}], 'payable': True, 'stateMutability': 'payable', 'type': 'function'}
{'inputs': [], 'payable': False, 'stateMutability': 'nonpayable', 'type': 'constructor'}
{'anonymous': False, 'inputs': [{'indexed': False, 'name': 'value', 'type': 'uint256'}], 'name': 'onSetValue', 'type': 'event'}
{'anonymous': False, 'inputs': [{'indexed': False, 'name': 'value', 'type': 'uint256'}], 'name': 'onGetValue', 'type': 'event'}
851
{"newAddress":"0de2540194e7673906da694620d3bb046ea0db64", "output":"608060405260043610610057576000357c01000000000000000000000000000000000000000000000

In [203]:
new_address

'0x1A24b0dF35034406122919d4E4162d4EbEd020b5'

In [156]:
web3.Web3.toChecksumAddress('36813e0c029e0ac04ba8da31a8b81d546db73481')

'0x36813E0c029E0AC04BA8Da31a8b81d546Db73481'

In [255]:
import web3
def pack_args():

    fn_identifier = 'setValue'
    fn_abi =  {'constant': False, 'inputs': [{'name': 'v', 'type': 'uint256'}], 'name': 'setValue', 'outputs': [], 'payable': False, 'stateMutability': 'nonpayable', 'type': 'function'}
    args = (119000,)
    kwargs = {}
    
    contract_abi =  [
      {'constant': False, 'inputs': [],                                 'name': 'getValue', 'outputs': [{'name': '', 'type': 'uint256'}],      'payable': False, 'stateMutability': 'nonpayable', 'type': 'function'}, 
      {'constant': False, 'inputs': [{'name': 'v', 'type': 'uint256'}], 'name': 'setValue', 'outputs': [],                                     'payable': False, 'stateMutability': 'nonpayable', 'type': 'function'}, 
      {'constant': True, 'inputs': [{'name': '', 'type': 'address'}],   'name': 'mymap',    'outputs': [{'name': '',  'type': 'uint256'}],     'payable': False, 'stateMutability': 'view',       'type': 'function'}, 
      {'inputs': [], 'payable': False,  'stateMutability': 'nonpayable', 'type': 'constructor'}, 
      {'anonymous': False, 'inputs': [{'indexed': False, 'name': 's', 'type': 'string'}], 'name': 'log', 'type': 'event'}
      ]
    
    data = web3._utils.contracts.encode_transaction_data(
            web3,
            fn_identifier,
            contract_abi,
            fn_abi,
            args,
            kwargs)
    return data

args = pack_args();
print(args)
transaction = {
     'from': '0x2c7536E3605D9C16a7a3D7b1898e529396a65c23',
     'to':new_address,
     'data': args,
     'gas': 2000000,
     'gasPrice': 23,
     'nonce': 0,
}

key = '0x4c0883a69102937d6231471b5dbb6204fe5129617082792ae468d01a3f362318'
signed = web3.eth.Account.signTransaction(transaction, key)
#print(signed.rawTransaction)
#for c in signed.rawTransaction:
#    print('0x%02x,'%(c,), end='')
name = 'helloworld11'
try:
    r = eosapi.push_action(name, 'exec', signed.rawTransaction, {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print(e)

0x55241077000000000000000000000000000000000000000000000000000000000001d0d8
{"newAddress":"0000000000000000000000000000000000000000", "output":""}
758


In [205]:
import web3

def pack_getValue():

    fn_identifier = 'getValue'
    fn_abi = {'constant': False, 'inputs': [],                                 'name': 'getValue', 'outputs': [{'name': '', 'type': 'uint256'}],      'payable': False, 'stateMutability': 'nonpayable', 'type': 'function'}
    args = ()
    kwargs = {}
    
    contract_abi =  [
      {'constant': False, 'inputs': [],                                 'name': 'getValue', 'outputs': [{'name': '', 'type': 'uint256'}],      'payable': False, 'stateMutability': 'nonpayable', 'type': 'function'}, 
      {'constant': False, 'inputs': [{'name': 'v', 'type': 'uint256'}], 'name': 'setValue', 'outputs': [],                                     'payable': False, 'stateMutability': 'nonpayable', 'type': 'function'}, 
      {'constant': True, 'inputs': [{'name': '', 'type': 'address'}],   'name': 'mymap',    'outputs': [{'name': '',  'type': 'uint256'}],     'payable': False, 'stateMutability': 'view',       'type': 'function'}, 
      {'inputs': [], 'payable': False,  'stateMutability': 'nonpayable', 'type': 'constructor'}, 
      {'anonymous': False, 'inputs': [{'indexed': False, 'name': 's', 'type': 'string'}], 'name': 'log', 'type': 'event'}
      ]
    
    data = web3._utils.contracts.encode_transaction_data(
            web3,
            fn_identifier,
            contract_abi,
            fn_abi,
            args,
            kwargs)
    return data

args = pack_getValue()
print(args)
transaction = {
     'from': '0x2c7536E3605D9C16a7a3D7b1898e529396a65c23',
     'to': new_address,
     'data': args,
     'gas': 2000000,
     'gasPrice': 23,
     'nonce': 0,
}

key = '0x4c0883a69102937d6231471b5dbb6204fe5129617082792ae468d01a3f362318'
signed = web3.eth.Account.signTransaction(transaction, key)
#print(signed.rawTransaction)
#for c in signed.rawTransaction:
#    print('0x%02x,'%(c,), end='')
name = 'helloworld11'
try:
    args = signed.rawTransaction.hex()[2:]
    r = eosapi.push_action(name, 'exec', signed.rawTransaction, {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print('exception:', e)

0x20965255
{"newAddress":"0000000000000000000000000000000000000000", "output":"000000000000000000000000000000000000000000000000000000000001d0d8"}
762


In [155]:
'%x'%(119000,)

'1d0d8'

In [161]:
bytes.fromhex('29325e60ff39fad605f9bb4f2fd7cc0ee8e778b3f52cb561c2d8eebfcec4ca4b')

b')2^`\xff9\xfa\xd6\x05\xf9\xbbO/\xd7\xcc\x0e\xe8\xe7x\xb3\xf5,\xb5a\xc2\xd8\xee\xbf\xce\xc4\xcaK'

In [230]:
import web3

def pack_getValue():

    fn_identifier = 'getValue'
    fn_abi = {'constant': False, 'inputs': [],                                 'name': 'getValue', 'outputs': [{'name': '', 'type': 'uint256'}],      'payable': False, 'stateMutability': 'nonpayable', 'type': 'function'}
    args = ()
    kwargs = {}
    
    contract_abi =  [
      {'constant': False, 'inputs': [],                                 'name': 'getValue', 'outputs': [{'name': '', 'type': 'uint256'}],      'payable': False, 'stateMutability': 'nonpayable', 'type': 'function'}, 
      {'constant': False, 'inputs': [{'name': 'v', 'type': 'uint256'}], 'name': 'setValue', 'outputs': [],                                     'payable': False, 'stateMutability': 'nonpayable', 'type': 'function'}, 
      {'constant': True, 'inputs': [{'name': '', 'type': 'address'}],   'name': 'mymap',    'outputs': [{'name': '',  'type': 'uint256'}],     'payable': False, 'stateMutability': 'view',       'type': 'function'}, 
      {'inputs': [], 'payable': False,  'stateMutability': 'nonpayable', 'type': 'constructor'}, 
      {'anonymous': False, 'inputs': [{'indexed': False, 'name': 's', 'type': 'string'}], 'name': 'log', 'type': 'event'}
      ]
    
    data = web3._utils.contracts.encode_transaction_data(
            web3,
            fn_identifier,
            contract_abi,
            fn_abi,
            args,
            kwargs)
    return data

args = pack_getValue()
print(args)
transaction = {
     'from': '0x2c7536E3605D9C16a7a3D7b1898e529396a65c23',
     'to': new_address,
     'data': args,
     'gas': 2000000,
     'gasPrice': 23,
     'nonce': 0,
}

key = '0x4c0883a69102937d6231471b5dbb6204fe5129617082792ae468d01a3f362318'
signed = web3.eth.Account.signTransaction(transaction, key)
#print(signed.rawTransaction)
#for c in signed.rawTransaction:
#    print('0x%02x,'%(c,), end='')
name = 'helloworld11'
try:
    args = signed.rawTransaction.hex()[2:]
    r = eosapi.call_contract(name, 'exec', args)
    res = r['results']['output']
    res = json.loads(res)
    print(res['output'])
    res = bytes.fromhex(res['output'])
    print(res[0:32])
    print(res[32:64])
    print(res[64:])
except Exception as e:
    print('exception:', e)

0x20965255
0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000b68656c6c6f2c776f726c64000000000000000000000000000000000000000000
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 '
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0b'
b'hello,world\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [212]:
result = bytes.fromhex('0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000b68656c6c6f2c776f726c64000000000000000000000000000000000000000000')

In [213]:
result[:64]

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0b'

In [215]:
result[64:128]

b'hello,world\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [218]:
len(result)

96

In [181]:
eosapi.s2n('helloworld11')

7684014468695212560

In [122]:
code = r'''
#include <eosio/print.hpp>
#include <eosio/name.hpp>
#include <eosio/action.hpp>
#include <eosio/multi_index.hpp>
#include <eosio/singleton.hpp>
#include <eosio/fixed_bytes.hpp>
#include "eth_account.hpp"

using namespace eosio;
using namespace std;

__attribute__((eosio_wasm_import))
extern "C" int evm_execute(const unsigned char* code, size_t size);

template<typename T>
T unpack_args() {
    size_t raw_args_size = action_data_size();
    check(raw_args_size > 0, "bad args");
    vector<char> raw_args(raw_args_size);
    read_action_data(raw_args.data(), raw_args_size);
    T rec = eosio::unpack<T>(raw_args);
    return rec;
}
   
extern "C" {
    void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
        eosio::print("hello,worlddddddd\n");
        return;
        if (action == "active"_n.value) {
        } else if (action == "exec"_n.value) {
        }
    }
}
#include "eth_account.cpp"
 
'''

abi = '''
{
   "version": "eosio::abi/1.0",
   "types": [],
  "structs": [{
      "name": "active",
      "base": "",
      "fields": [
        {"name":"address", "type":"bytes"},
       ]
    }
  ],
  "actions": [{
      "name": "active",
      "type": "active",
      "ricardian_contract": ""
    },{
      "name": "exec",
      "type": "bytes",
      "ricardian_contract": ""
    }
  ],
  "tables": [
  ]
}
'''

publish_cpp_contract('helloworld11', code, abi, includes=['contracts'])
print('done!')




done!


In [123]:
eosapi.call_contract(name, 'exec', '112233')

{
    "results": {
        "output": "hello,worlddddddd\n"
    }
}